In [16]:
from svd_compression import decompose_layers
from training_utils.trainer import Trainer

import torch
from torch.utils.data import random_split, DataLoader

from torchvision.datasets import ImageFolder
from torchvision import transforms

from transformers import AutoModelForImageClassification

In [17]:
ImageNetValidationTransformation = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet mean
        std=[0.229, 0.224, 0.225],   # ImageNet std
    ),
])

dataset = ImageFolder(
    root='tiny-imagenet-200/train', 
    transform=ImageNetValidationTransformation
)

train_dataset, val_dataset, test_dataset = random_split(dataset, [0.8, 0.1, 0.1])

In [41]:
import pickle

with open('imgnet200_datasets.pkl', 'wb') as file:
    pickle.dump((train_dataset, val_dataset, test_dataset), file)

In [18]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=True)

In [13]:
device = 'cuda:4'
model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")
model.load_state_dict(torch.load('ViTimgnet200_0.86acc.pt'))


num_params_wo_compression = sum(p.numel() for p in model.parameters())
model = decompose_layers(model, threshold=1, max_rank=192)
num_params_with_compression = sum(p.numel() for p in model.parameters())

print(f'Compression ratio: {num_params_with_compression / num_params_wo_compression}')

model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=1000)

trainer = Trainer(
    model, 
    train_loader,
    val_loader, 
    optimizer, 
    scheduler, 
    device
)

{'initial_num_of_weights': 86567656, 'post_svd_num_of_weights': 47639272, 'after / before': 0.5503126017412323}
Gotcha!
Compression ratio: 0.5503126017412323


In [18]:
train_history, val_history = trainer.run(num_epochs=3)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

AttributeError: 'Trainer' object has no attribute 'validation_step'

In [19]:
trainer.val_step(k_for_top_k=5)

  0%|          | 0/40 [00:00<?, ?it/s]

{'accuracy': 0.81210935,
 'accuracy@5': 0.89384765625,
 'loss': 0.780474791303277}